In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import xarray as xr
from scipy.interpolate import interp1d

amsu_channel9_df = pd.read_csv('../AMSU_channel9_2022.csv')
total = amsu_channel9_df.Weight.sum()
amsu_channel9_df = amsu_channel9_df[amsu_channel9_df['Height'] > 7]
amsu_channel9_df = amsu_channel9_df[amsu_channel9_df['Height'] < 41]
#plt.plot(amsu_channel9_df.Weight, amsu_channel9_df.Height)
weighting_function_heights = amsu_channel9_df.Height
wf_interp = interp1d(amsu_channel9_df.Height, amsu_channel9_df.Weight, kind='cubic')
wf_heights_100m_spacing = np.linspace(8,40,161)
wf_weights_100m_spacing = wf_interp(wf_heights_100m_spacing)
wf_heights_100m_spacing = wf_heights_100m_spacing*1000


In [5]:
amsu_channel9_df

,Height,Weight
13,40.74819,0.00199
14,39.30693,0.00308
15,37.95080,0.00465
16,36.67055,0.00684
17,35.45842,0.00987
18,34.30776,0.01394
19,33.21282,0.01925
20,32.16868,0.02602
21,31.17099,0.03464
22,30.21597,0.04543


## First find synthetic TLS measurements based on the ROM SAF CDR

In [ ]:
ROM_SAF_path = '/home/bdc2/aodhan/ROM_SAF/www.romsaf.org/pub/cdr/v1.0/profs/grace/atm/'

for year in range(2007, 2017):
    RO_daily_data_paths = glob.glob(ROM_SAF_path + str(year) + '/*')
    yearly_TLS_data = []
    for day in RO_daily_data_paths:
        datafiles_of_day = glob.glob(day + '/*')
        for datafile in datafiles_of_day:
            if datafile[-11:] == 'non-nominal':
                continue
            else:
                RO = xr.open_dataset(datafile)
                dry_temp = RO.dry_temp.values[0] - 273.15
                alt = RO.alt_refrac.values[0] 
                # above is the height according to page 51/51 at url below
                # https://www.romsaf.org/product_documents/romsaf_pum.pdf 
                if alt[0] <= 8000: # Make sure ROs have data to find TLS
                    altm17 = np.abs(alt-17000)
                    idx_17km = np.nanargmin(altm17) # 17 Km tangent point
                    lat = RO.lat_tp.values[0][idx_17km]
                    lon = RO.lon_tp.values[0][idx_17km]
                    time = RO.time.values[0]
                    month = RO.month.values[0]

                    # interpolate temperature profile
                    temp_interp = interp1d(alt, dry_temp, kind='cubic')
                    temp_100m_spacing = temp_interp(wf_heights_100m_spacing)
                    weighted_temp = np.multiply(temp_100m_spacing, wf_weights_100m_spacing)
                    summed_weighted_temp = np.nansum(weighted_temp)
                    summed_weight = np.nansum(wf_weights_100m_spacing)

                    # find TLS value
                    TLS = summed_weighted_temp/summed_weight

                    data_row = [month, lat, lon, time, TLS]
                    yearly_TLS_data.append(data_row)
                else:
                    continue            
    np.save(ROM_SAF_path + str(year) + '/grace_TLS_measurements.npy', yearly_TLS_data)
    

# Next Find TLS measurements based on ROM SAF ICDR

In [3]:
ROM_SAF_path = '/home/bdc2/aodhan/ROM_SAF/www.romsaf.org/pub/icdr/v1-series/profs/metop/atm/'

for year in range(2022, 2023):
    RO_daily_data_paths = glob.glob(ROM_SAF_path + str(year) + '/*')
    yearly_TLS_data = []
    for day in RO_daily_data_paths:
        datafiles_of_day = glob.glob(day + '/*')
        for datafile in datafiles_of_day:
            if datafile[-11:] == 'non-nominal':
                continue
            else:
                RO = xr.open_dataset(datafile)
                dry_temp = RO.dry_temp.values[0] - 273.15
                alt = RO.alt_refrac.values[0] 
                # above is the height according to page 51/51 at url below
                # https://www.romsaf.org/product_documents/romsaf_pum.pdf 
                if alt[0] <= 8000: # Make sure ROs have data to find TLS
                    altm17 = np.abs(alt-17000)
                    idx_17km = np.nanargmin(altm17) # 17 Km tangent point
                    lat = RO.lat_tp.values[0][idx_17km]
                    lon = RO.lon_tp.values[0][idx_17km]
                    time = RO.time.values[0]
                    month = RO.month.values[0]

                    # interpolate temperature profile
                    temp_interp = interp1d(alt, dry_temp, kind='cubic')
                    temp_100m_spacing = temp_interp(wf_heights_100m_spacing)
                    weighted_temp = np.multiply(temp_100m_spacing, wf_weights_100m_spacing)
                    summed_weighted_temp = np.nansum(weighted_temp)
                    summed_weight = np.nansum(wf_weights_100m_spacing)

                    # find TLS value
                    TLS = summed_weighted_temp/summed_weight

                    data_row = [month, lat, lon, time, TLS]
                    yearly_TLS_data.append(data_row)
                else:
                    continue            
    np.save(ROM_SAF_path + str(year) + '/metop_TLS_measurements.npy', yearly_TLS_data)
    

In [13]:
RO.time_offset

<xarray.DataArray 'time_offset' (dim_unlim: 1)>
array([105747322083], dtype='timedelta64[ns]')
Dimensions without coordinates: dim_unlim
Attributes:
    long_name:    Time offset for georeferencing (since start of occ.)
    valid_range:  [-10.    239.999]